In [17]:
# %%
# Import packages and libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pydotplus
from scipy.stats import randint

# from patsy import dmatrices, dmatrix
import random
import re
# import pickle
import statsmodels.api as sm
from sklearn.linear_model import lars_path
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.externals.six import StringIO

from IPython.display import Image, display
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense, Activation
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import resample
from sklearn.utils import shuffle

seed = 5
np.random.seed(seed)

In [8]:
reduced = pd.read_csv("loan_reduced.csv", low_memory=False)

important = reduced[['term', 'interest', 'grade',
                          'othdebt_inc', 'loan_inc', 
                          'loan_status']]

# Decision Tree Chart

In [2]:
loan_impt = pd.read_csv("loan_impt.csv", low_memory=False)

In [5]:
# Validation on different linear models
kf = KFold(n_splits=5, shuffle=True, random_state = np.random.seed(seed))
X, y = loan_impt.drop(["loan_status"],
                    axis=1), loan_impt["loan_status"]

# hold out 20% of the data for final testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state= np.random.seed(seed))
# # further partition X, y into datasets X_train, y_train (60% of original) and X_val, y_val (20%)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=np.random.seed(seed))

In [6]:
#%%
def quick_test(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)
    model.fit(xtrain, ytrain)
    return model.score(xtest, ytest)

def quick_test_afew_times(model, X, y, n=5):
    return np.mean([quick_test(model, X, y) for j in range(n)])

In [7]:
# Using with Decision Tree
decisiontree = DecisionTreeClassifier(max_depth=2)
# Do the test 10 times with a Decision Tree and get the average score
quick_test_afew_times(decisiontree, X_test, y_test)

0.726248688775922

In [8]:
#%%
# Tree classifier graph

dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,max_depth = 5,
                special_characters=True, feature_names=X_train.columns,precision = 2)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

Image(graph.create_png())
graph.write_png('sampletree.png') # Write it to a file

True

## Reference for Parameter Tuning##

In [18]:
import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# get some data
digits = load_digits()
X, y = digits.data, digits.target

# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

# start = time()
random_search.fit(X, y)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# # use a full grid over all parameters
# param_grid = {"max_depth": [3, None],
#               "max_features": [1, 3, 10],
#               "min_samples_split": [2, 3, 10],
#               "bootstrap": [True, False],
#               "criterion": ["gini", "entropy"]}

# # run grid search
# grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, iid=False)
# start = time()
# grid_search.fit(X, y)

# print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
#       % (time() - start, len(grid_search.cv_results_['params'])))
# report(grid_search.cv_results_)

Model with rank: 1
Mean validation score: 0.931 (std: 0.025)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 7, 'min_samples_split': 8}



## KNN##

In [55]:
# build a classifier
clf = KNeighborsClassifier()

# specify parameters and distributions to sample from

param_dist = {"n_neighbors": range(7, 9, 2),
                  "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
                  "leaf_size":  randint(30, 100),
                  "p": randint(1, 2)}



# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

# start = time()
random_search.fit(X_train, y_train)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


KeyboardInterrupt: 

## DecisionTree##

In [57]:
# build a classifier
clf = DecisionTreeClassifier()

# specify parameters and distributions to sample from

param_dist = {"max_depth": [3, None],
              "max_features": range(3,6),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}


# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


RandomizedSearchCV took 57.37 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.728 (std: 0.001)
Parameters: {'criterion': 'gini', 'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 2}



## GaussianNB##


In [59]:
# build a classifier
clf = GaussianNB()

# specify parameters and distributions to sample from

param_dist = {"priors": [None]}


# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=20. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV took 0.43 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.711 (std: 0.001)
Parameters: {'priors': None}



## Bernoulli NB##

In [64]:
# build a classifier
clf = BernoulliNB()

# specify parameters and distributions to sample from

param_dist = {"alpha": np.arange(0,1),
                  "fit_prior": [True]}


# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=20. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\naive_bayes.py:955: RuntimeWarning: divide by zero encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\naive_bayes.py:958: RuntimeWarning: invalid value encountered in add
  jll += self.class_log_prior_ + neg_prob.sum(axis=1)
c:\users\regina\appdata\local\programs\python\python37\lib\si

RandomizedSearchCV took 0.44 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.723 (std: 0.000)
Parameters: {'fit_prior': True, 'alpha': 0}



c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


## LogisticRegression##

In [53]:
# build a classifier
clf = LogisticRegression(C = 0.5)

# specify parameters and distributions to sample from
param_dist = {"penalty": ["l2", "none"],
                     "tol": [0.2,0.4,0.6,0.8],  # tolerance
                     "C": [0.2,0.4,0.6,0.8],# for regularization penalty
                     #"class_weight": {0: 0.3, 1: 0.7}, # How shd i tune this for higher recall?
                     "solver": ["newton-cg", "lbfgs", "sag", "saga"],
                     "multi_class": ["auto"],
                     #"verbose": 1,
                     "l1_ratio": [0.2,0.4,0.6,0.8]}




# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

# start = time()
random_search.fit(X_train, y_train)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\reg

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\reg

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\reg

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\lo

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
c:\users\reg

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
c:\users\regina\appdata\loca

Model with rank: 1
Mean validation score: 0.730 (std: 0.001)
Parameters: {'tol': 0.8, 'solver': 'saga', 'penalty': 'none', 'multi_class': 'auto', 'l1_ratio': 0.6, 'C': 0.2}



## LinearSVC##

In [70]:

# build a classifier
clf = LinearSVC()

# specify parameters and distributions to sample from
param_dist = {#"penalty": ["l1", "l2"],
                           "loss": ["hinge" ,"squared_hinge" ],
                           "dual": [True, False]}




# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 4
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\regina\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\regina\appdata\local\programs\python\python37\lib\site-

ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

## Bagging LinearSVC with Logistic Regression## Parameters?

In [ ]:

# build a classifier
clf = baggit_logit()

# specify parameters and distributions to sample from
param_dist = {"penalty": ["l1", "l2"],
                           "loss": ["hinge" ,"squared_hinge" ],
                           "dual": [True, False],
                           "max_iter": 20}




# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

# start = time()
random_search.fit(X_train, y_train)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

## RandomForestClassifier##

In [19]:
# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=1):  # specify how many scores to print
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": (3, 5),
              "min_samples_split": randint(2, 11),
              "criterion": ["gini", "entropy"],
              "bootstrap": [True, False]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

# start = time()
random_search.fit(X_train, y_train)
# print("RandomizedSearchCV took %.2f seconds for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


KeyboardInterrupt: 

In [ ]:
random_search.best_estimator_.predict_proba(X_test)

In [ ]:
print(random_search.best_estimator_.predict_proba(X_test)[:,1])

In [ ]:
y_pred = random_search.best_estimator_.predict_proba(X_test)[:,1]

In [ ]:
f1_score(X_test, y_pred)

## Search for Optimal Threshold -> Classification Matrix for KNN##

In [10]:
#KNN
knn = KNeighborsClassifier(n_neighbors=7,weights='distance',algorithm='auto')
knn_best = knn.fit(X_train, y_train)
y_pred = knn_best.predict(X_test)
# print(quick_test_afew_times(knn_best, X_test, y_test))
# print("Log-loss on knn: {:6.4f}".format(log_loss(y_test, knn.predict_proba(X_test))))
# # Classification Report 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.85      0.80     44921
           1       0.40      0.26      0.32     17043

    accuracy                           0.69     61964
   macro avg       0.58      0.56      0.56     61964
weighted avg       0.66      0.69      0.67     61964



In [11]:
print('\nSearch for OPTIMAL THRESHOLD, vary from 0.001 to 0.999, fit/predict on train/test data')
score_list = []
th_list = []
for th in 10**np.linspace(-3,-0.0001,200):   # 0.001 to 0.999
   knn.fit(X_train, y_train)
   y_pred = (knn.predict_proba(X_test)[:,1] >= th)
   score_list.append(f1_score(y_test, y_pred))
   th_list.append(th)
   print('{:.4f}'.format(f1_score(y_test, y_pred)), end=', ')   # 4 decimal pl
print('optimal F1 score = {:.4f}'.format(max(score_list)))
optimal_th = float(th_list[score_list.index(max(score_list))])
print('optimal threshold = {:.4f}'.format(optimal_th))


Search for OPTIMAL THRESHOLD, vary from 0.001 to 0.999, fit/predict on train/test data
0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4502, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4502, 0.4503, 0.4503, 0.4503, 0.4503, 0.4503, 0.4504, 

In [12]:
#%%
def make_confusion_matrix(model, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    fraud_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(fraud_confusion, cmap=plt.cm.Purples, annot=True, square=True, fmt='d',
           xticklabels=['Fully Paid', 'Default'],
           yticklabels=['Fully Paid', 'Default'])
    plt.xlabel('Prediction')
    plt.ylabel('Actual')

#%%
from ipywidgets import interactive, FloatSlider

interactive(lambda threshold: make_confusion_matrix(knn, threshold), threshold=(0.0,1.0,0.01))


interactive(children=(FloatSlider(value=0.5, description='threshold', max=1.0, step=0.01), Output()), _dom_cla…